# Explore here

In [11]:
import pandas as pd 
import re

In [7]:
#EXPORTAR DATOS 
df=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
df.head()
df.columns

Index(['url', 'is_spam'], dtype='object')

In [10]:
#CAMBIO TRUE=1, FALSE=0
df["is_spam"]= df["is_spam"].astype(int)
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [32]:
df.dtypes
df["url"].isna().sum()


np.int64(0)

In [33]:
df["url"].astype(str)

0       ['https://briefingday.us8.list-manage.com/unsu...
1                              ['https://www.hvper.com/']
2                  ['https://briefingday.com/m/v4n3i4f3']
3       ['https://briefingday.com/n/20200618/m#comment...
4                         ['https://briefingday.com/fan']
                              ...                        
2994    ['https://www.smartcitiesworld.net/news/news/d...
2995                    ['https://www.youtube.com/watch']
2996    ['https://techcrunch.com/2019/07/04/an-optimis...
2997    ['https://www.technologyreview.com/2019/12/20/...
2998     ['https://www.bbc.com/news/technology-51018758']
Name: url, Length: 2999, dtype: object

In [31]:
def preparacion(text):
    text= re.sub(r"\s+[a-z]\s+", " ", text)
    return text.split()

df["url"]= df["url"].apply(preparacion)
df.head()

TypeError: expected string or bytes-like object, got 'list'